In [1]:
# libraries
from dotenv import load_dotenv
import os
from tavily import TavilyClient

# load environment variables from .env file
_ = load_dotenv()

# connect
client = TavilyClient(api_key=os.environ.get("TAVILY_API_KEY"))

In [2]:
# run search
result = client.search("What is in Nvidia's new Blackwell GPU?",
                       include_answer=True)

# print the answer
result["answer"]


'The new Blackwell GPU uses TSMC 4NP node, GDDR7 memory, and features advanced Tensor Cores and NVLink interconnect.'

## Regular search

In [3]:
# choose location (try to change to your own city!)

city = "San Francisco"

query = f"""
    what is the current weather in {city}?
    Should I travel there today?
    "weather.com"
"""

In [4]:
import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
import re

ddg = DDGS()

def search(query, max_results=6):
    try:
        results = ddg.text(query, max_results=max_results)
        return [i["href"] for i in results]
    except Exception as e:
        print(f"returning previous results due to exception reaching ddg.")
        results = [ # cover case where DDG rate limits due to high deeplearning.ai volume
            "https://weather.com/weather/today/l/USCA0987:1:US",
            "https://weather.com/weather/hourbyhour/l/54f9d8baac32496f6b5497b4bf7a277c3e2e6cc5625de69680e6169e7e38e9a8",
        ]
        return results  


for i in search(query):
    print(i)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_6720\2852339428.py:6: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  ddg = DDGS()


https://zhidao.baidu.com/question/10035040.html
https://www.current-news.co.uk/nztc-awards-500000-to-clean-tech-startups/
https://www.current-news.co.uk/milence-expands-ev-truck-charging-in-germany-france/
https://www.current-news.co.uk/energy-systems-catapult-launches-smart-data-service-to-speed-net-zero-journey/
https://www.current-news.co.uk/cornwall-insight-levy-exemptions-could-slash-bills-by-25-for-small-manufacturers/
https://www.current-news.co.uk/fastned-opens-renewable-energy-powered-north-east-england-ev-charging-hub/


In [5]:
def scrape_weather_info(url):
    """Scrape content from the given URL"""
    if not url:
        return "Weather information could not be found."
    
    # fetch data
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return "Failed to retrieve the webpage."

    # parse result
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup


In [6]:
# use DuckDuckGo to find websites and take the first result
url = search(query)[0]

# scrape first wesbsite
soup = scrape_weather_info(url)

print(f"Website: {url}\n\n")
print(str(soup.body)[:50000]) # limit long outputs

Website: https://zhidao.baidu.com/question/10035040.html


<body class="layout-center has-menu">
<div class="userbar userbar-renew fixed qb-search" data="" id="userbar" left-width="880" width="" win-min-width="1280">
<ul class="aside-list">
<li>
<a class="toindex" href="http://www.baidu.com/">ç¾åº¦é¦é¡µ</a>
</li>
<li class="shop-entrance">
<a href="/shop" log="type:2026,pos:top-right,target:shop-entrance" title="ç¥éåå">åå<i class="i-house" style="display: none;"></i></a>
<span class="lucky-try" style="display: none"></span>
</li>
<li><a alog-alias="usrbar-reg" href="https://passport.baidu.com/v2/?reg&amp;tpl=ik&amp;color=green&amp;u=" id="userbar-reg" rel="nofollow" target="_blank">æ³¨å</a></li>
<li><a alog-alias="usrbar-login" href="javascript:;" id="userbar-login" log="type:2026,pname:account,mod:login,action:show,pos:pop" rel="nofollow">ç»å½</a></li>
</ul>
</div>
<div class="head-wrap">
<header class="container" id="header">
<div class="search-box-new line search-bo

In [7]:
# extract text
weather_data = []
for tag in soup.find_all(['h1', 'h2', 'h3', 'p']):
    text = tag.get_text(" ", strip=True)
    weather_data.append(text)

# combine all elements into a single string
weather_data = "\n".join(weather_data)

# remove all spaces from the combined text
weather_data = re.sub(r'\s+', ' ', weather_data)
    
print(f"Website: {url}\n\n")
print(weather_data)

Website: https://zhidao.baidu.com/question/10035040.html


è¯·é®ä¸ºä»ä¹çµåçè±ææ¯voltage,ç¼©åå´æ¯uï¼èçµæµè±ææ¯current,ç¼©åæ¯I? è¯·é®ä¸ºä»ä¹çµåçè±ææ¯voltage,ç¼©åå´æ¯uï¼èçµæµè±ææ¯current,ç¼©åæ¯I? 10 å¯éä¸­1ä¸ªæå¤ä¸ªä¸é¢çå ³é®è¯ï¼æç´¢ç¸å ³èµæãä¹å¯ç´æ¥ç¹âæç´¢èµæâæç´¢æ´ä¸ªé®é¢ã å ¶ä»ç±»ä¼¼é®é¢ ä¸ºä½ æ¨èï¼ ç¹å«æ¨è ç­ä½ æ¥ç­ ç±»å« æä»¬ä¼éè¿æ¶æ¯ãé®ç®±ç­æ¹å¼å°½å¿«å°ä¸¾æ¥ç»æéç¥æ¨ã è¯´æ 0 /200 æçè´¢å¯å¼ 0 -- æçç°é 0 ä¸è½½ç¾åº¦ç¥éAPP å¨APPç«¯-ä»»å¡ä¸­å¿æç° æç¥éäº -- ç´¯è®¡å®æ ä¸ªä»»å¡ 10ä»»å¡ ç¥ç¥ç¥ç¥â¦ 50ä»»å¡ ç¥ç¥ç¥ç¥â¦ 100ä»»å¡ ç¥ç¥ç¥ç¥â¦ 200ä»»å¡ ç¥ç¥ç¥ç¥â¦ æ¨çè´¦å·ç¶ææ­£å¸¸ æè°¢æ¨å¯¹æä»¬çæ¯æ äº¬ICPè¯030173å·-1 äº¬ç½æã2023ã1034-029å· ©2025Baidu ä½¿ç¨ç¾åº¦åå¿ è¯» | ç¥éåè®® | ä¼ä¸æ¨å¹¿ è¾ å© æ¨¡ å¼


In [8]:
# run search
result = client.search(query, max_results=1)

# print first result
data = result["results"][0]["content"]

print(data)

{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1758033449, 'localtime': '2025-09-16 07:37'}, 'current': {'last_updated_epoch': 1758033000, 'last_updated': '2025-09-16 07:30', 'temp_c': 15.6, 'temp_f': 60.1, 'is_day': 1, 'condition': {'text': 'Fog', 'icon': '//cdn.weatherapi.com/weather/64x64/day/248.png', 'code': 1135}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 95, 'wind_dir': 'E', 'pressure_mb': 1013.0, 'pressure_in': 29.91, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 96, 'cloud': 0, 'feelslike_c': 15.6, 'feelslike_f': 60.1, 'windchill_c': 13.4, 'windchill_f': 56.2, 'heatindex_c': 13.1, 'heatindex_f': 55.6, 'dewpoint_c': 13.1, 'dewpoint_f': 55.5, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 6.7, 'gust_kph': 10.8}}


In [9]:
import json
from pygments import highlight, lexers, formatters

# parse JSON
parsed_json = json.loads(data.replace("'", '"'))

# pretty print JSON with syntax highlighting
formatted_json = json.dumps(parsed_json, indent=4)
colorful_json = highlight(formatted_json,
                          lexers.JsonLexer(),
                          formatters.TerminalFormatter())

print(colorful_json)


{
    "location": {
        "name": "San Francisco",
        "region": "California",
        "country": "United States of America",
        "lat": 37.775,
        "lon": -122.4183,
        "tz_id": "America/Los_Angeles",
        "localtime_epoch": 1758033449,
        "localtime": "2025-09-16 07:37"
    },
    "current": {
        "last_updated_epoch": 1758033000,
        "last_updated": "2025-09-16 07:30",
        "temp_c": 15.6,
        "temp_f": 60.1,
        "is_day": 1,
        "condition": {
            "text": "Fog",
            "icon": "//cdn.weatherapi.com/weather/64x64/day/248.png",
            "code": 1135
        },
        "wind_mph": 2.2,
        "wind_kph": 3.6,
        "wind_degree": 95,
        "wind_dir": "E",
        "pressure_mb": 1013.0,
        "pressure_in": 29.91,
        "precip_mm": 0.0,
        "precip_in": 0.0,
        "humidity": 96,
        "cloud": 0,
        "feelslike_c": 15.6,
        "feelslike_f": 60.1,
        "windchill_c": 13.4,
        "windchill_